In [1]:
import math # cos() for Rastrigin
import copy # array-copying convenience
import sys # max float
import numpy as np
import sympy as sp
from sympy.plotting import plot
from sympy.plotting import plot3d
import matplotlib.pyplot as plt
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
from ddpg_agent import Agent
import matplotlib.pyplot as plt
from scipy.optimize import dual_annealing

In [2]:
## parameter/ constant
bandwidth = 1e6  # [hz]
Speed_of_light = 3e8  # [m/s]
Carrier_freq = 2e9  # [hz]=1/s
GBS_location = [0,0,20] #[m] 20
Num_GBS = 1
# NUM_User = 10
NUM_UAV = 2
Cell_Radius = 1000 #[m]
UAV_Height = 100#100 #[m] "Cell-Edge User Offloading via Flying UAV in Non-Uniform Heterogeneous Cellular Networks," in IEEE Transactions on Wireless Communications, vol. 19, no. 4, pp. 2411-2426, April 2020, doi: 10.1109/TWC.2020.2964656.
P_UAV = 10 **(37 / 10) / 1000 #[watt] 
P_GBS = 10 **(40 / 10) / 1000 #[watt]
P_MAX = 0.4 # 26dbm, user
# P_MIN = 0.1 # 20 dbm
epsilon = 0.38 # power amplifier efficiency
AWGN_DBM = -174 #[dBm]
AWGN_W = 10 ** (AWGN_DBM / 10) / 1000
air_density = 1.225 # Air density at sea level (kg/m³)
g = 9.81            # Gravitational acceleration (m/s²)
mass = 0.5           # Mass of UAV in kg
radius = 0.2         # Rotor radius in meters
num_rotors = 4
P_MOVE =10 #[watt]
V = 20 # m/s


In [3]:
#Area
AREA_WIDTH = 1000 #[m]
def random_position():
    # Generate a random position within the area
    x = np.random.uniform(-Cell_Radius, Cell_Radius)
    y = np.random.uniform(-Cell_Radius, Cell_Radius)
    return (x, y, 0)

User_position = []
for i in range(45):
    User_position.append(random_position())

UAV_2D = []
for i in range(2):
    UAV_2D.append(random_position())

UAV_position=[]
for i in range(2):
    # Convert to a list to modify it
    temp_list = list(UAV_2D[i])
    temp_list[2] = UAV_Height
    updated_tuple = tuple(temp_list)
    UAV_position.append(updated_tuple)

In [4]:
User_position

[(-39.57804365814411, -419.3110588372235, 0),
 (-714.8002067891609, 222.5864888245028, 0),
 (-739.6672231978907, -973.3426135565908, 0),
 (-20.401671945591943, -573.8284824931488, 0),
 (-305.06126123786953, 41.76485583057797, 0),
 (-977.8928807574587, 67.77027350881508, 0),
 (37.85542505615058, 529.8429516325552, 0),
 (-49.50122593894582, -774.522430569268, 0),
 (806.4529406197257, -143.29132010367493, 0),
 (-177.72373988034394, 181.06785872265777, 0),
 (-610.8645644848006, -820.8516307718671, 0),
 (633.0894260712389, 538.4606604479575, 0),
 (-440.07520593117124, -345.7290823286419, 0),
 (385.24023993243486, 434.56504205150577, 0),
 (16.925794202204315, -436.5584844579964, 0),
 (-703.8564898848036, -588.3103932521235, 0),
 (-575.5608613278356, 652.1464190988879, 0),
 (173.81548132642206, 429.539507927748, 0),
 (362.5929950371785, -216.65795084284343, 0),
 (482.1922732567923, -556.2364820120808, 0),
 (-892.5464508287155, -86.66038579128178, 0),
 (-323.8057841861637, -958.1634400385333, 

In [5]:
def init_uav_positions(NUM_UAV, UAV_Height, random_position):

    UAV_2D = [random_position() for _ in range(NUM_UAV)]
    UAV_positions = []
    
    for i in range(NUM_UAV):
        temp_list = list(UAV_2D[i])
        temp_list[2] = UAV_Height
        updated_tuple = tuple(temp_list)
        UAV_positions.append(updated_tuple)
    
    return UAV_positions

In [6]:
# initail location UAVs
UAV_position =np.array(UAV_position)
UAV_position

array([[ 695.84220776, -584.03276239,  100.        ],
       [-300.59982575,  271.26413994,  100.        ]])

In [7]:
# Fixed location User
User_position = [
(653.8316581607805, -590.3141344883956, 0),
 (655.7839795019095, 511.39715579708627, 0),
 (-366.9200930533958, -378.53106855942656, 0),
 (774.0710077290566, -813.74371579118, 0),
 (-902.1513920204236, 689.4058695488745, 0),
 (-887.8742394199635, 629.5122712145035, 0),
 (-480.4469889568361, -411.7900064526441, 0),
 (-584.0565231366488, -207.29087777276175, 0),
 (917.8907259745158, -388.6065319450123, 0),
 (482.22239122176074, 382.7717524589923, 0),
 (75.08205947,	-246.6113445,	0),
(89.08034796,	93.40797308	,0),
(539.8534823,	-160.3015493,	0),
(139.7089773,	716.6716587	,0),
(-569.3566777,	263.1666404	,0),
(-90.29446879,	-288.6084637	,0),
(-767.5899487,	-135.3924119,	0),
(773.8435759	,-235.8694596	,0),
(613.2944812	,608.7494777	,0),
(204.2851089	,-171.0078052	,0),
# (386.1443989	,-926.5117843	,0),
# (-938.0904915	,99.43896713	,0),
# (-937.1604717,	-444.6725165	,0),
# (694.8819289	,668.5820704	,0),
# (834.6212365	,987.1367586,	0),
# (348.2927666	,-81.11493568	,0),
# (-955.8690815	,81.17473736	,0),
# (-545.2280657	,977.3220921	,0),
# (-700.3981729	,-579.959419	,0),
# (653.7133739	,-669.164471	,0),
]

In [8]:
NUM_User = len(User_position)
NUM_User

20

In [9]:
#Calculate Distance
def distance(x1,y1,z1,x2,y2,z2):
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
D_n_bs = np.zeros((NUM_UAV + Num_GBS, NUM_User))
D_n_bs

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [10]:
#pathloss, unitless
def gain_n(distance):
    c = Speed_of_light
    f= Carrier_freq
    pi = np.pi
    return (c/(4*pi*f*distance))**(2)

In [11]:
#Generte link selection
def generate_matrix_and_assign_users(rows=3, cols=10):
    while True:
        matrix_selection = np.zeros((rows, cols), dtype=int)
        for col in range(cols):
            row_idx = np.random.randint(0, rows)
            matrix_selection[row_idx, col] = 1
        
        # Calculate row sums
        tmp = np.sum(matrix_selection, axis=1)
        if tmp[0] > 0 and tmp[1] > 0:
            break
    
    # Determine user assignments based on the matrix
    UAV1_user = np.argwhere(matrix_selection[0] == 1).T[0].tolist()
    UAV2_user = np.argwhere(matrix_selection[1] == 1).T[0].tolist()
    GBS_user = np.argwhere(matrix_selection[2] == 1).T[0].tolist()
    
    return matrix_selection, UAV1_user, UAV2_user, GBS_user
matrix_selection, UAV1_user, UAV2_user, GBS_user = generate_matrix_and_assign_users()

In [12]:
# generate selectionn not yet set constraint
import random
rows = 3
cols = 10
while True:
    matrix_selection = np.zeros((rows, cols), dtype=int) # matrix
    for col in range(cols):
        row_idx = np.random.randint(0, rows)
        matrix_selection[row_idx, col] = 1
    tmp = np.sum(matrix_selection, axis=1)
    # print(tmp[0])
    if tmp[0] > 0 and tmp[1] > 0:
        break
UAV1_user = np.argwhere(matrix_selection[0] == 1).T[0].tolist()
UAV2_user = np.argwhere(matrix_selection[1] == 1).T[0].tolist()
GBS_user = np.argwhere(matrix_selection[2] == 1).T[0].tolist()

In [13]:
# # SINR with NOMA in same coverage of UAV1
# num_tslot =24
# time_slots = np.arange(0, num_tslot)
power_min = 0.1
power_max = P_UAV

p_u1_level = np.random.uniform(power_min, power_max, len(UAV1_user))
p_u1_normalize = p_u1_level / np.sum(p_u1_level) * power_max
power_sort = sorted(p_u1_normalize)
power_sort

[0.21334529736671276,
 0.3524799804677603,
 0.4467179564366162,
 0.6446879967707939,
 1.6161129963005083,
 1.7385281089303324]

In [14]:
# SINR
def noma_sinr(channel_conditions, selected_users, powers, transmitter_id):
    if not selected_users:
        return [], []
    
    gains = [channel_conditions[transmitter_id, user] for user in selected_users]
    # Sort users by channel gain (strongest first)
    sorted_indices = np.argsort(gains)[::-1]  # Descending order
    sorted_users = [selected_users[i] for i in sorted_indices]
    sorted_gains = [gains[i] for i in sorted_indices]
    sinrs = []
    for i, user in enumerate(sorted_indices):
        interference = 0
        sinr = (powers[user] * sorted_gains[user]) / (AWGN_W + interference)
        sinrs.append(sinr)
    
    return sinrs, selected_users
sinrs = noma_sinr(gain_n(D_n_bs), UAV1_user, power_sort, 0)
sinrs

c:\Users\Sothearath\anaconda3\envs\python3_7\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  


([inf, inf, inf, inf, inf, inf], [2, 3, 4, 5, 6, 9])

In [15]:
#Data rate
num_user = NUM_User
sorted_users = sinrs[1]
sorted_users_sinr = sinrs[0]
def calculate_transmission_rate(sinr, num_user):
    """Calculate achievable rate using Shannon capacity"""
    if num_user == 0:
        num_user = 1
    return (int(bandwidth/num_user) * np.log2(1 + sinr))/1e6 # MHz

rate_n = calculate_transmission_rate(np.array(sorted_users_sinr), 1)
rate_n

array([inf, inf, inf, inf, inf, inf])

In [16]:
#Data rate from transformer peak 16PM
# 10 user
# traffic_data = np.array([ 8.795845 ,  4.476116 ,  5.5037837,  6.0243654, 12.920729 ,
#         5.5970373,  6.0550256,  4.872516 ,  7.577386 ,  9.967498 ])
# # #15 user
# traffic_data = np.array([14.2345, 4.7710037, 7.6004195, 7.6562877, 22.904835, 10.31743,
#                          10.02901, 10.140568, 13.525869, 17.060368, 20.13406, 3.3490102, 
#                          17.419847, 15.829944, 6.5886827 ])
# 20 user
# traffic_data = np.array([15.045018, 6.545699, 7.0056453, 7.0192256, 26.408241, 9.260441, 
#                          10.931942, 8.048492, 12.15611, 15.898077, 19.099787, 4.9521737,
#                            17.933542, 14.467922, 9.955734, 13.156038, 21.471113, 10.216565, 8.602531, 17.791065])

#25 user
# traffic_data = np.array([14.101378, 6.327937, 6.9734707, 7.705143, 24.17083, 8.45224, 10.937698,
#                           7.165816, 13.620338, 16.299187, 21.602406, 4.3687615, 18.788303, 16.177567, 
#                           8.267505, 15.565486, 23.637806, 11.268315, 8.559526, 16.202307, 17.567959, 23.138298, 
#                           19.848198, 4.852388, 10.035355])

# 30 user
traffic_data = np.array([13.730608, 5.326337, 6.305037, 8.3221855, 24.156334, 9.056003, 11.426689, 11.308687, 
                         13.707179, 16.739891, 22.561272, 4.2011957, 16.53558, 13.905987, 4.317765, 14.405717, 
                         23.871864, 11.336344, 9.031543, 19.897724, 16.41292, 19.279444, 18.578587, 5.886368, 8.510113,
                           8.288387, 10.033254, 18.78224, 12.353946, 24.130177])

In [17]:
len(traffic_data)

30

In [18]:
action_raw = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0 ,1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0 ]  


In [19]:
len(action_raw)

48

In [20]:
# Scale from DDPG to actual value
def actual_power(x):
    return (x + 1) * (0.4 / 2) #0.4

def actual_band_ratio(x):
    return (np.array(x) + 1) / 2

def actual_location(x):
    return (x * 1000)

def actual_selected(x):
    return ((x + 1) * (NUM_UAV + Num_GBS - 1))/2
actual_selected(action_raw[-1])

2.0

In [21]:
# Find max Rate
max_band_ratio = 1
distance_min = 100 #meter
P_MAX_user = 0.5011872336272725 
max_gain = gain_n(distance_min)
max_sinr = ( P_MAX_user* max_gain) / (AWGN_W )
max_rate = (max_band_ratio * bandwidth) * np.log2(1 + max_sinr)/1e6 # MHz
max_rate

40.70611888041029

In [22]:
#power constrains for (-1,1)

# Define the max power for the UAV
P_max = P_UAV 
num_users = NUM_User
P_min = 0.001  
P_max_user = 0.5011872336272725  

# Define the scaling function to map DDPG actions (-1 to 1) to power allocations
def scale_action_to_power(action, P_min, P_max_user):
    return (action + 1) / 2 * (P_max_user - P_min) + P_min

# Function to ensure total power does not exceed P_max
def control_power_allocation(actions, P_max):
    power_allocations = scale_action_to_power(actions, P_min, P_max_user)
    total_power = np.sum(power_allocations)
    
    # Step 3: If total power exceeds P_max, scale down allocations proportionally
    if total_power > P_max:
        scaling_factor = P_max / total_power
        power_allocations = power_allocations * scaling_factor
        
    return power_allocations

# Example DDPG action (output from the DDPG agent)
ddpg_actions = np.random.uniform(-1, 1, num_users)  # Random actions for testing

# Control the power allocation
allocated_power = control_power_allocation(ddpg_actions, P_max)
allocated_power


array([0.43232622, 0.1302291 , 0.45655955, 0.25912875, 0.23012548,
       0.39527968, 0.48050516, 0.04259215, 0.28829959, 0.17302914,
       0.13928386, 0.09440731, 0.10411798, 0.32899133, 0.12344397,
       0.39625365, 0.0825437 , 0.46301945, 0.04641166, 0.27760828])

In [23]:
# Define the max power for the UAV
P_max = P_UAV 
num_users = NUM_User
P_min = 0.001  
P_max_user = 0.5011872336272725  
def control_power_allocation(actions, P_max):
    power_allocations = actions
    total_power = np.sum(actions)
    
    # Step 3: If total power exceeds P_max, scale down allocations proportionally
    if total_power > P_max:
        scaling_factor = P_max / total_power
        power_allocations = power_allocations * scaling_factor
        
    return power_allocations
actions = np.array([0.5,1,3,2])
# control_power_allocation(actions, P_max)


In [24]:
# Calculate Energy consumption
num_tslot = 24
time_slots = np.arange(0, num_tslot)
def calculate_com_energy(time_slot,p_tx, data_tx, data_rate):
   
    opt_time = data_tx / data_rate 
    
    # energy_hov = (hover_power * max(opt_time))  # Watt-s
    energy_com = (p_tx * opt_time)           # Watt-s
    return energy_com ,0 #energy_hov

In [25]:
# Scale UAV: Objective function 
request_rate = traffic_data
def object_function(action):
    P_max = P_UAV
    power_level = np.array([action[i] for i in range(NUM_User)])
    # power_allocations = control_power_allocation(power_level[:NUM_User], P_max)
    power_allocations = power_level
    # print(f"power_allocations:{(power_allocations)}")
    uav_locations = np.array([(action[i]) for i in range(NUM_User, NUM_User + 2 * NUM_UAV)])
    
    uav_positions = uav_locations.reshape(NUM_UAV, 2)  # Reshape into (NUM_UAV, 2) -> (x, y) pairs
    
    end_index = min(NUM_User + 2 * NUM_UAV + NUM_User, len(action))    
   
    link_values = action[NUM_User + 2 * NUM_UAV : end_index]
    
    link_uavs = {uav_idx: [] for uav_idx in range(NUM_UAV)}
    link_bs = []
    bs_user_count = 0  # Variable to track the number of users connected to the BS

    for i, value in enumerate(link_values):
        user_assigned = False
        
        # First, try to assign the user to a UAV if possible
        for uav_idx in range(NUM_UAV):
            if uav_idx <= value < (uav_idx + 1):
                link_uavs[uav_idx].append(i)
                user_assigned = True
                break
        
        # If the user wasn't assigned to a UAV, check the number of users at the BS
        if not user_assigned:
            if bs_user_count < 5:  # If there are less than 15 users at the BS, assign to BS
                link_bs.append(i)
                bs_user_count += 1
            else:  # If there are more than 15 users, assign the user to a UAV
                # Find an appropriate UAV (you can modify this logic to be more dynamic if needed)
                # For simplicity, let's assign the user to the first UAV
                # link_uavs[0].append(i)  # This could be adjusted for load balancing
                # print(f"==>>link_uav1: {(len(link_uavs[0]))}")
                # print(f"==>>link_uav2: {(len(link_uavs[1]))}")
                min_uav_idx = min(range(NUM_UAV), key=lambda uav_idx: len(link_uavs[uav_idx]))
                link_uavs[min_uav_idx].append(i)

    # print(f"bs_user_count:{bs_user_count}")
    # print(f"link_uavs:{link_uavs}")
    # print(f"link_bs:{link_bs}")
    powers = power_allocations
    p_user_uavs = {uav_idx: powers[link_uavs[uav_idx]] for uav_idx in range(NUM_UAV)}
    
    link_lists = [link_uavs[i] for i in range(NUM_UAV)]
    p_user_lists = [p_user_uavs[i][0:len(link_lists[i])] for i in range(NUM_UAV)]
    p_user_bs = powers[link_bs]

    for uav_id in range(NUM_UAV):
        x_uav, y_uav = uav_positions[uav_id]
        for j in range(NUM_User):
            user_pos = User_position[j]
            D_n_bs[uav_id][j] = distance(x1=x_uav, y1=y_uav, z1=UAV_Height, x2=user_pos[0], y2=user_pos[1], z2=user_pos[2])

    for j in range(NUM_User):
        user_pos = User_position[j]
        D_n_bs[NUM_UAV][j] = distance(x1=GBS_location[0], y1=GBS_location[1], z1=GBS_location[2], x2=user_pos[0], y2=user_pos[1], z2=user_pos[2])

    channel_gains = gain_n(D_n_bs)
    
    sinr_uavs = [noma_sinr(channel_gains, link_lists[i], p_user_lists[i], i) for i in range(NUM_UAV)]
    sinr_bs = noma_sinr(channel_gains, link_bs, p_user_bs, NUM_UAV)
    rate_mbps_uavs = [calculate_transmission_rate(np.array(sinr_uavs[i][0]), len(link_lists[i])) for i in range(NUM_UAV)]
    rate_mbps_bs = calculate_transmission_rate(np.array(sinr_bs[0]), len(link_bs))
    # sum_rate = sum([np.sum(rate) for rate in rate_mbps_uavs]) + np.sum(rate_mbps_bs)
    requested_rates_uavs = [request_rate[sinr_uavs[i][1]] for i in range(NUM_UAV)]
    energy_uavs = [calculate_com_energy(time_slots[0], p_user_lists[i], requested_rates_uavs[i], rate_mbps_uavs[i]) if link_lists[i] else ([0], [0]) for i in range(NUM_UAV)]
    
    energy_com_uavs = [energy_uavs[i][0] for i in range(NUM_UAV)]
    energy_hov_uavs = [0 for _ in range(NUM_UAV)]
    energy_mov_uavs = [0.00 for _ in range(NUM_UAV)]
    
    sum_energy_uavs = [energy_mov_uavs[i] + np.sum(energy_com_uavs[i]) + energy_hov_uavs[i] for i in range(NUM_UAV)]
    sum_energy = sum(sum_energy_uavs)
    check_minrate = [*rate_mbps_bs] + [rate for sublist in rate_mbps_uavs for rate in sublist]
    UAV_energy = sum_energy_uavs
    

    avg_rate = np.mean(check_minrate)
    avg_energy = sum_energy / NUM_UAV
    max_energy = P_MAX_user * 5
    rate_normalize = avg_rate * (1 / int(max_rate))
    energy_normalize = - (avg_energy / max_energy)
    result_obj_normalize = (0.5*rate_normalize) +  (0.5*energy_normalize)
    # print(f"==>>result_obj_normalize: {(result_obj_normalize)}")
    return result_obj_normalize , avg_rate, avg_energy
    
object_function(action_raw)


(-14.162553724678723, 1.8204913778488687, 71.09496210338183)

In [26]:
# only reward
final_result = []
final_action = []
# for _ in range (10): 
def obj_f(x): 
    result_obj_normalize, _, _ = object_function(x)
    return - result_obj_normalize

bounds = [(P_min, P_max_user),(P_min, P_max_user),(P_min, P_max_user), (P_min, P_max_user),(P_min, P_max_user), (P_min, P_max_user), (P_min, P_max_user),(P_min, P_max_user),(P_min, P_max_user), (P_min, P_max_user),
        (P_min, P_max_user), (P_min, P_max_user),(P_min, P_max_user),(P_min, P_max_user), (P_min, P_max_user),(P_min, P_max_user), (P_min, P_max_user),(P_min, P_max_user),(P_min, P_max_user), (P_min, P_max_user),
        
        (-1000,1000), (-1000,1000), (-1000,1000), (-1000,1000),(-1000,1000), (-1000,1000),(-1000,1000), (-1000,1000),
         (0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3),(0,3)]
results = {'fun': [], 'x': [], 'avg_rate':[], 'avg_energy':[]}
# Define the callback function to print results at each iteration
def callback(xk, f, context):
    results['fun'].append(f)
    results['x'].append(xk)
    # Call object_function to get rate and energy
    result_obj_normalize, avg_rate, avg_energy = object_function(xk)
    results['avg_rate'].append(avg_rate)
    results['avg_energy'].append(avg_energy)

    # Print directly at each iteration
    print(f"Iteration {len(results['x'])}: Reward={result_obj_normalize:.4f} | Rate={avg_rate:.4f} | Energy={avg_energy:.4f}")
# result = dual_annealing(obj_f, bounds, callback=callback, maxiter=100)
try:
    result = dual_annealing(obj_f, bounds, callback=callback, maxiter=1000)
except Exception as e:
    print(f"An error occurred: {e}")

final_action.append(result['x'])
final_result.append(result['fun'])
# Initialize an empty dictionary to store results
print("\nOptimization Result:")
print(f"Final f(x) = {result['fun']:.4f}")
print(f"Final x = {result['x']}")


# plt.plot(np.negative((results['fun'])))
# plt.xlabel('Iteration')
# plt.ylabel('f(x)')
# plt.title('Reward Trade-off')
# plt.show()

Iteration 1: Reward=-0.8964 | Rate=4.8401 | Energy=4.7959
Iteration 2: Reward=-0.8223 | Rate=4.9173 | Energy=4.4293
Iteration 3: Reward=-0.7901 | Rate=4.9095 | Energy=4.2672
Iteration 4: Reward=-0.7198 | Rate=4.9460 | Energy=3.9175
Iteration 5: Reward=0.0479 | Rate=4.3428 | Energy=0.0319
Iteration 6: Reward=0.0479 | Rate=4.6030 | Energy=0.0481
Iteration 7: Reward=0.0479 | Rate=4.6030 | Energy=0.0481
Iteration 8: Reward=0.0479 | Rate=4.6030 | Energy=0.0481
Iteration 9: Reward=0.0479 | Rate=4.6033 | Energy=0.0481
Iteration 10: Reward=0.0479 | Rate=4.6033 | Energy=0.0481
Iteration 11: Reward=0.0480 | Rate=4.6046 | Energy=0.0479
Iteration 12: Reward=0.0508 | Rate=4.5486 | Energy=0.0305

Optimization Result:
Final f(x) = -0.0508
Final x = [ 5.01187234e-01  5.01187234e-01  5.01187234e-01  5.01187234e-01
  1.00000000e-03  1.00000000e-03  1.00000000e-03  1.00000000e-03
  1.00000000e-03  1.00000000e-03  1.00000000e-03  5.01187234e-01
  1.00000000e-03  1.00000000e-03  1.24595638e-03  1.00000000e